# 23S rRNA Discovery

The purpose of this experiment is to extract all 23S rRNA related data from NCBI/Genbank files. The 23S rRNA is another conserved region in bacteria we can attempt to use for designing scientific assays to detect the specific region as it is thought of as having conserved and unique regions across all bacterial species; allowing us to detect and discriminate between different bacterias within a sample.

Biopython will be used to interact with the Entrez API to extract a list of predetermined bacterial species we want. The dataset used for this experiment will be for the vaginal microbiome, approximately 900+ species have been identified residing in the vaginal microbiome according to past studies such as ones done for the analysis of the causitive mechanism of bacterial vaginosis.

### Data Exploration

Let's read in the data

In [ ]:
import os

# SETTINGS
""" Directory Structure

../downloads/<date of data>

"""
DIRECTORY_MAIN = 'downloads'
DIRECTORY_DATE = 'test'
DIRECTORY_SAVE = 'results'
DIRECTORY_PATH = os.path.join(DIRECTORY_MAIN, DIRECTORY_DATE)
DIRECTORY_SAVE_PATH = os.path.join(DIRECTORY_MAIN, DIRECTORY_DATE, DIRECTORY_SAVE)

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

# Holds a list of SeqRecord ojects for writing 
seq_record_list = []

def parse_record(genbankfile):
    """ Reads the Genbank Files 
    
    Args:
        genbankfile - *.gbk file
    Returns:
        seq_location - gene of interest
    """
    
    # Iterates over a single Genbank file; can contain multiple Genbank entries per file
    for record in SeqIO.parse(genbankfile, 'gb'):
        parse_features(record)

def parse_features(genbankrecord):
    
    """ Reads in the entire seq feature
    and extracts only the ones we want; in this case
    the 23S rRNA feature.
    
    Args:
        genbankrecord (SeqRecord) - individual genbank record
    """
    
    INTERESTED_FEATURE = 'rRNA'
    # If any type of 23S rRNA is found, this is marked as True
    FEATURE_PRESENT = False
    
    # Iterates over a single Genbank entry's features
    for items in genbankrecord.features:
        if items.type == 'rRNA':
            try:
                # The type of rRNA is most of the time classified under 'product' or 'gene'
                if (items.qualifiers['product'][0].lower().find('23s')>-1 or
                    items.qualifiers['gene'][0].lower().find('23s')>-1):
                    
                    FEATURE_PRESENT = True
                    
                    # Create the SeqRecord Object
                    query_seq = items.extract(genbankrecord.seq)
                    query_id = genbankrecord.name
                    query_name = genbankrecord.annotations['organism'].replace(" ","_")
                    seq_create = SeqRecord(query_seq, id=query_id, description=query_name)
                    seq_record_list.append(seq_create)
             
            # Some entries may not have any entries for product; could be formatting of entries
            # Currently the one sure way to determine the rRNA type is 23S
            except KeyError:
#                 print("Error, no rRNA product.", items, genbankrecord.name)
                continue
    
    # If sequence is present, write out the sequences
    if FEATURE_PRESENT:
        write_to_fasta(seq_record_list, query_name)
    
def write_to_fasta(seq_list, save_as):
    """ Creates and writes out a SeqRecord Object
    to FASTA format
    
    Args:
        query_seq (List of SeqRecords) - write to FASTA
        save_as (str) - name of FASTA file to save as
    """
    
    import re
    
    # For removing special characters in file name
    pattern = re.compile("[^\w.\s]")
    file_name = pattern.sub('', save_as)
    file_name += ".fasta"
    
    save_dir = os.path.join(os.getcwd(), DIRECTORY_SAVE_PATH, file_name)
    SeqIO.write(seq_list, save_dir, 'fasta')
 
        


In [ ]:
def run(directory):
    """ Read all of the files within the directory.
    
    Args:
        directory - the directory containing the genbank files
    """
    
    for files in os.listdir(directory):
        file_dir = os.path.join(DIRECTORY_PATH, files)
        parse_record(file_dir)

run(DIRECTORY_PATH)